<a href="https://colab.research.google.com/github/ArchitGupta16/Topic-Modeling/blob/main/Pipeline_Anvil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
!pip install transformers
!pip install -U sentence-transformers
!pip install hdbscan
!pip install umap-learn
!pip install anvil-uplink

import anvil.server
import anvil.media
import matplotlib.pyplot as plt
import umap.umap_ as umap
import pandas as pd
from sentence_transformers import SentenceTransformer
import umap.umap_ as um
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

anvil.server.connect("server_UVYNGR7VHA6XPK4JVKXPQYIC-43YI7Q4JG4YWXLGL")

In [ ]:
import chardet
from tqdm import tqdm
import spacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
@anvil.server.callable
def file_reader(file):
  with open('filename', 'wb') as f:
        f.write(file.get_bytes())

    # Create a Media object from the uploaded file
  media_object = anvil.media.from_file('filename')

  # Read the contents of the media object
  data = media_object.get_bytes()
  print(data)
  # encoding = chardet.detect(data)['encoding']
  data_text = data.decode()
  print(data_text)
  return data_text

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

with open('/content/gdrive/MyDrive/Colab Notebooks/Summary/umap_model_summary.pkl', 'rb') as f:
      umap_embeddings = pickle.load(f)

with open('/content/gdrive/MyDrive/Colab Notebooks/Summary/dbscan_model_summary.pkl', 'rb') as f:
    cluster = pickle.load(f)



Mounted at /content/gdrive


In [ ]:
def c_tf_idf(documents, m, ngram_range=(1, 1)):
      count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
      t = count.transform(documents).toarray()
      w = t.sum(axis=1)
      tf = np.divide(t.T, w)
      sum_t = t.sum(axis=0)
      idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
      tf_idf = np.multiply(tf, idf)

      return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
      words = count.get_feature_names_out()
      labels = list(docs_per_topic.Topic)
      tf_idf_transposed = tf_idf.T
      indices = tf_idf_transposed.argsort()[:, -n:]
      top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
      return top_n_words

def extract_topic_sizes(df):
      topic_sizes = (df.groupby(['Topic']).Doc.count().reset_index().rename({"Topic": "Topic", "Doc": "Size"}, axis='columns').sort_values("Size", ascending=False))
      return topic_sizes


In [ ]:
@anvil.server.callable
def topic_modelling(inp):

  l = inp.split("@@@@@")
  # for line in inp:
  #   l.append(line)
  # print(len(l))
  print(l)
  print(type(l),len(l))
  model = SentenceTransformer('distilbert-base-nli-mean-tokens')
  embeddings = model.encode(l, show_progress_bar=True)
  # print(embeddings)

  new_embeddings= umap_embeddings.transform(embeddings)

  labels_test = cluster.fit_predict(new_embeddings)


  umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
  result = pd.DataFrame(umap_data, columns=['x', 'y'])
  result['labels'] = cluster.labels_

  # Visualize clusters
  fig, ax = plt.subplots(figsize=(20, 10))
  outliers = result.loc[result.labels == -1, :]
  clustered = result.loc[result.labels != -1, :]
  plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
  plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')

  #@title c TF-IDF

  docs_df = pd.DataFrame(l, columns=["Doc"])
  docs_df['Topic'] = cluster.labels_
  docs_df['Doc_ID'] = range(len(docs_df))
  docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})


  tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(l))
  top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
  topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

  print(type(top_n_words[-1][:10]),top_n_words[-1][:10])

  doc_pos = {}


  for i in tqdm(range(len(l))):
    document = nlp(l[i])
    # Token and Tag
    temp = {}

    for token in document:
      a = str(token)
      if token.pos_=="PROPN":
        temp[a] = "PROPN"


      elif token.pos_=="NOUN":
        temp[a] = "NOUN"

      elif token.pos_=="VERB":
        temp[a] = "VERB"

      else:
        temp[a] = "OTHER"

      doc_pos[i]=temp

  tem=[]
  for i in doc_pos.items():

    for j,k in i[1].items():

      if k == 'NOUN' or k == 'PROPN':
        tem.append(j)
    #if i[1]=='NOUN':
      #print(i[0])
  # print(tem)

  topics = {}
  t = []

  for i in range(-1,len(top_n_words)-1):
    for j in range(0,20):
      if top_n_words[i][j][0] in tem:
        t.append(top_n_words[i][j])
    topics[i] = t
    t = []

  output = ""
  for i in topics[-1]:
    print(i[0])
    output += " " + i[0]
  print(output)
  return output

In [ ]:
anvil.server.wait_forever()
